In [ ]:
import os 
from dotenv import load_dotenv
from typing import List, Dict

In [1]:
# Dummy initial data for users making use of the app
favorite_genres: List = [
    "Science Fiction",
    "Fantasy",
    "Mystery",
    "Romance",
    "Horror",
]

reading_history: List[Dict] = [
    {"title": "Dune", "author": "Frank Herbert", "genre": "Science Fiction"},
    {"title": "The Hobbit", "author": "J.R.R. Tolkien", "genre": "Fantasy"},
    {"title": "The Da Vinci Code", "author": "Dan Brown", "genre": "Mystery"},
    {"title": "Pride and Prejudice", "author": "Jane Austen", "genre": "Romance"},
]

user_bio: Dict = {
    "name": "Alice Smith",
    "age": 29,
    "location": "New York, USA",
    "occupation": "Software Developer",
}   

In [4]:
_ = load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [5]:
system_prompt = """
You are LitSense, an AI-powered literary recommendation engine. 
Your task is to provide personalized book recommendations based on user preferences, and reading history.
Your other task (more important) is to serve as an aid to make informed buying decisions for users looking to purchase books from a physical or online bookstore.
Users can provide you with images of book covers, and you will analyze these images to extract relevant information such as title, author, and genre, also if it fits the user reading preference. 
Use the following guidelines to generate your responses:
1. Analyze the user's favorite genres and reading history to understand their literary tastes.
2. Analyze book cover images provided by the user to extract title, author, and genre information.
3. When given a cover image, determine if the book aligns with the user's interests and recommend accordingly.
4. Recommend books that align with the user's interests, ensuring a mix of popular titles and hidden gems.
5. Provide a brief summary of each recommended book, highlighting why it would appeal to the user
"""

In [8]:
import base64
def encode_image_to_base64(image_path: str) -> str:
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string

In [11]:
image_path = "assets/book cover 2.jpg"
encoded_image = encode_image_to_base64(image_path)

In [12]:
user_prompt = f"""
Analyze the following user data:
Favorite Genres: {favorite_genres}
Reading History: {reading_history}
User Age: {user_bio['age']}
User Location: {user_bio['location']}
User Occupation: {user_bio['occupation']}

You are provided with an image of a book cover, extract the title, author, and genre from the image, and determine if it fits the user's reading preferences.
If it fits, recommend the book to the user with a brief summary explaining why it would appeal to them.
""" 

In [14]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-4.1",
    input=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": user_prompt},
                {"type": "input_image",
                 "image_url": f"data:image/jpeg;base64,{encoded_image}"}
            ]
        }
    ]
)

print(response.output_text)

**Extracted Book Information:**

- **Title:** Harry Potter and the Cursed Child (Parts One and Two)
- **Authors:** J.K. Rowling, John Tiffany, & Jack Thorne
- **Genre:** Fantasy / Script (Theatrical Play)

---

**Fit with Your Preferences:**
You enjoy **Fantasy**, and your reading history includes "The Hobbit"—indicating a fondness for imaginative worlds, magic, and epic storytelling. "Harry Potter and the Cursed Child" is an extension of the beloved Harry Potter universe, making it an excellent fit for your interests.

---

**Recommendation & Brief Summary:**

**Harry Potter and the Cursed Child** picks up the story nineteen years after the original series ended, focusing on Harry Potter’s son, Albus Severus Potter, as he wrestles with his family’s legacy and the dark shadows that linger from the past. Filled with magic, intrigue, friendship, and new mysteries, this script offers a fresh and emotional journey through the wizarding world. 

**Why You’d Enjoy It:**
- **Continuation of a